In [ ]:
!pip install graphlearning

In [2]:
import numpy as np
import pandas as pd
import graphlearning as gl
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.linalg import eigh
import networkx as nx

In [3]:
W = pd.read_csv("/kaggle/input/da324dataminingproject2/adjacency.csv")
W.head()

,0,1,2,3,4,5,6,7,8,9,...,11942,11943,11944,11945,11946,11947,11948,11949,11950,11951
0,0,1,0,0,1,1,0,0,0,0,...,1,1,1,1,1,1,0,0,1,1
1,1,0,0,0,1,0,1,1,0,1,...,1,1,1,1,1,1,1,0,1,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,1,1,1,0,0,0,0,1
3,0,0,0,0,1,0,0,0,1,1,...,1,0,0,0,0,1,0,1,1,1
4,1,1,0,1,0,0,0,0,1,1,...,1,1,1,0,1,1,0,1,1,1


In [4]:
W.shape

(11952, 11952)

In [5]:
X = pd.read_excel("/kaggle/input/da324dataminingproject2/attributes.xlsx")
X.head()

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,-6.3128,-5.58510,-3.0679,12.9880,9.00260,-5.5361,-1.0356,10.1650,-8.6108,4.3228,...,-2.8601,-24.768,18.2130,7.3551,44.382,18.659,-6.0441,2.9410,29.0100,-32.906
1,1.3902,-0.44661,-3.7137,-12.8420,4.47870,4.8796,3.0049,-5.9097,-7.3532,13.8250,...,-29.5170,-35.920,7.0253,-12.0390,43.115,-71.516,-25.2070,-6.4823,96.8440,-67.232
2,-8.3044,-3.45690,6.5153,-12.0090,-6.76200,-1.2466,-5.9627,9.9527,-6.2527,12.3210,...,13.8550,17.979,-25.7940,16.4910,13.304,-25.527,-52.4930,79.9770,8.9831,30.085
3,-6.3335,-5.83730,-1.5020,-5.0176,6.49150,-25.8940,11.6040,5.1414,-2.5690,6.2234,...,35.8110,18.597,-27.5800,12.5700,10.744,-38.588,10.7530,2.9410,29.0100,-32.906
4,5.6422,-2.65020,4.1389,-5.6228,0.74712,2.7475,-6.3798,9.0944,-13.5540,10.9460,...,7.1254,-38.921,-67.4780,29.1870,27.917,13.592,19.5940,31.2670,7.3044,-84.121


In [6]:
model = gl.clustering.spectral(W, num_clusters=10)
pred_labels = model.fit_predict()

pred_labels


KeyboardInterrupt



In [ ]:
import networkx as nx
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Create a graph from the adjacency matrix
G = nx.Graph()
print(1)
edges = np.transpose(np.nonzero(W))
print(2)
G.add_edges_from(edges)
print(3)

# Define a function to compute centrality measures for a given node
def compute_centrality(node):
    degree = G.degree(node)
    eigenvector_cent = nx.eigenvector_centrality(G, max_iter=1000)[node]
    betweenness_cent = nx.betweenness_centrality(G)[node]
    clustering_coeff = nx.clustering(G, node)
    return node, degree, eigenvector_cent, betweenness_cent, clustering_coeff

# Use ThreadPoolExecutor for parallel computation
with ThreadPoolExecutor() as executor:
    centrality_results = list(executor.map(compute_centrality, G.nodes()))

# Extract results
node_degrees = {node: degree for node, degree, _, _, _ in centrality_results}
eigenvector_centralities = {node: eigenvector for node, _, eigenvector, _, _ in centrality_results}
betweenness_centralities = {node: betweenness for node, _, _, betweenness, _ in centrality_results}
clustering_coefficients = {node: clustering for node, _, _, _, clustering in centrality_results}

# Add new columns to the feature dataframe
X[103] = [node_degrees[node] for node in X.index]
X[104] = [eigenvector_centralities[node] for node in X.index]
X[105] = [betweenness_centralities[node] for node in X.index]
X[106] = [clustering_coefficients[node] for node in X.index]

1
2
3


In [ ]:
X

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


X = attributes_with_clusters.iloc[:, :-1]
y = attributes_with_clusters['Cluster']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1000, random_state=42)

nan_indices = y_train.index[y_train.isna()]
X_train = X_train.drop(index=nan_indices)
y_train = y_train.drop(index=nan_indices)

mlp_classifier = MLPClassifier(hidden_layer_sizes=(
    100,), activation='relu', solver='adam', random_state=42)
mlp_classifier.fit(X_train, y_train)

predicted_labels = mlp_classifier.predict(X_test)

In [54]:
sub = pd.DataFrame(pred_labels).reset_index()
sub = sub.rename(columns = {'index': 'ID', 0: 'LABEL'})
sub

,ID,LABEL
0,0,9
1,1,4
2,2,8
3,3,6
4,4,2
...,...,...
11947,11947,1
11948,11948,1
11949,11949,0
11950,11950,4


In [8]:
X.shape

(11952, 103)

In [9]:
seed = pd.read_excel("/kaggle/input/da324dataminingproject2/seed.xlsx")
seed.head()

,5793,10805,2258
0,8938,7423,1853
1,4566,902,5633
2,1529,9833,7709
3,1942,935,4907
4,6345,10950,2174


In [10]:
seed.shape

(9, 3)

In [ ]:
# Create a graph from the adjacency matrix
G = nx.Graph()
edges = np.transpose(np.nonzero(W))
G.add_edges_from(edges)

# # Draw the graph
# nx.draw(G, with_labels=False, node_color='skyblue')

# # Show the plot
# plt.show()

In [ ]:
# Create a graph from the adjacency matrix
G = nx.Graph()
edges = np.transpose(np.nonzero(W))
G.add_edges_from(edges)


# 1. Node Degree
node_degree = dict(G.degree())

# 2. Node Centrality
# 2.1 Eigenvector Centrality 

# 2.2 Betweenness Centrality
betweenness_centrality = nx.betweenness_centrality(G)

# 3. Clustering Coefficient
clustering_coefficient = nx.clustering(G)

# 4. Graphlets (Graphlet Degree Vector - GDV)
# Function to compute GDV for a given node
def compute_gdv(graph, node):
    gdv = nx.graphlets.graphlet_degree_vector(graph, node)
    return gdv

# Create lists to store feature values
node_degrees = [node_degree[node] for node in X.index]
eigenvector_centralities = [eigenvector_centrality[node] for node in X.index]
betweenness_centralities = [betweenness_centrality[node] for node in X.index]
clustering_coefficients = [clustering_coefficient[node] for node in X.index]
# gdvs = [compute_gdv(G, node) for node in X.index]

# Add new columns to the feature dataframe
X[103] = node_degrees
X[104] = eigenvector_centralities
X[105] = betweenness_centralities
X[106] = clustering_coefficients

# # Convert GDV lists to DataFrame and merge with X
# gdv_df = pd.DataFrame(gdvs, index=X.index)
# X = pd.concat([X, gdv_df], axis=1)

In [ ]:
X.to_csv('X.csv', index = False)

In [9]:
model = gl.clustering.spectral(W, num_clusters=3)
pred_labels = model.fit_predict()

pred_labels

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([1, 2, 1, ..., 0, 1, 1], dtype=int32)

In [ ]:
# Elbow method to find optimal number of clusters
wcss = []  # Within cluster sum of squares

for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:
# Plotting the elbow method
plt.figure(figsize=(10, 6))
plt.plot(range(1, 20), wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')  # Within cluster sum of squares
plt.show()

# # From the plot, we can see the elbow point, suggesting the optimal number of clusters.
# # Let's choose that and perform clustering.
# optimal_num_clusters = 4

# # Applying k-means clustering with the optimal number of clusters
# kmeans = KMeans(n_clusters=optimal_num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
# kmeans.fit(X)

# # Visualizing the clusters
# plt.figure(figsize=(10, 6))
# plt.scatter(X[:, 0], X[:, 1], c=kmeans

In [ ]:
from sklearn.cluster import AffinityPropagation
import numpy as np

clustering = AffinityPropagation(random_state=5).fit(X)
clustering.labels_
# clustering.predict([[0, 0], [4, 4]])

In [13]:
sub = pd.DataFrame(clustering.labels_).reset_index()
sub = sub.rename(columns = {'index': 'ID', 0: 'LABEL'})
sub

,ID,LABEL
0,0,1
1,1,2
2,2,1
3,3,0
4,4,1
...,...,...
11947,11947,2
11948,11948,2
11949,11949,0
11950,11950,1


In [55]:
sub.to_csv('submission.csv', index = False)

In [ ]:
optimal_num_clusters = 8

# Applying k-means clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(X)